In [81]:
import json
import pandas as pd

## Data Loading

In [ ]:
with open('../data/message_1.json', encoding='utf-8') as f:
    messages_dict = json.load(f)

In [84]:
messages_dict.keys()

dict_keys(['participants', 'messages', 'title', 'is_still_participant', 'thread_path', 'magic_words', 'image', 'joinable_mode'])

In [88]:
#Function fixes wrong unicode encodings of cyrillic script
def cyrillic_decoder(bad_decoding):
    return bad_decoding.encode('latin1').decode('utf-8')

In [85]:
name_chat = cyrillic_decoder(messages_dict['title'])
members = [cyrillic_decoder(x['name']) for x in messages_dict['participants']]
messages_bad = messages_dict['messages']

In [89]:
ids_no_messages = []
count_no_messages = 0
for index, message in enumerate(messages_bad):
    if message.get('content','no_content') == 'no_content':
        count_no_messages +=1
        ids_no_messages.append(index)
print(count_no_messages)

971


## Preprocessing (Decode badly encoded characters to cyrillic)

In [ ]:
messages = []

for message in messages_bad:

    if message.get('content','no_content') != 'no_content':
        message_dict = {'sender_name': cyrillic_decoder(message['sender_name']),
                        'timestamp_ms': message['timestamp_ms'],
                        'content': cyrillic_decoder(message['content'])}
        messages.append(message_dict)


In [ ]:
messages_df = pd.DataFrame.from_dict(messages)